dictionary를 만들자

In [62]:
text=['dog like cat', 
      'dog dog fish river',
      'cat cute odd eye', 
      'cat dog human',
      'dog bark cut cry',
      'cat run dog run',
      'cat cat cat dog dog dog',
      'dog eat cat cat dog']

In [63]:
#단어 리스트
lst=[]
for i,j in enumerate(text):
    for a in j.split():
        lst.append(a)
print(lst)

['dog', 'like', 'cat', 'dog', 'dog', 'fish', 'river', 'cat', 'cute', 'odd', 'eye', 'cat', 'dog', 'human', 'dog', 'bark', 'cut', 'cry', 'cat', 'run', 'dog', 'run', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'eat', 'cat', 'cat', 'dog']


In [64]:
#lst를 set으로 집합
vocabulary=set(lst)
print(vocabulary)

{'cat', 'eye', 'cut', 'like', 'human', 'eat', 'cute', 'river', 'bark', 'odd', 'dog', 'run', 'fish', 'cry'}


dictionary 만들기

In [65]:
diction={}
for p, q in enumerate(vocabulary):
    diction[q]= p+1
print(diction)

{'cat': 1, 'eye': 2, 'cut': 3, 'like': 4, 'human': 5, 'eat': 6, 'cute': 7, 'river': 8, 'bark': 9, 'odd': 10, 'dog': 11, 'run': 12, 'fish': 13, 'cry': 14}


단어 수치화

In [66]:
#단어 리스트
voca_lst=[]
for txt in text:
    dic_lst=[]
    for word in txt.split():
        dic_lst.append(diction[word])
    voca_lst.append(dic_lst)
print(voca_lst)

[[11, 4, 1], [11, 11, 13, 8], [1, 7, 10, 2], [1, 11, 5], [11, 9, 3, 14], [1, 12, 11, 12], [1, 1, 1, 11, 11, 11], [11, 6, 1, 1, 11]]


제일 긴 문장에 맞추어 padding작업  
  
----->Encoding(텍스트를 단순히 수치화, 단어-문장 의미는 전혀 고려 X  
-수치데이터는 단어나 문장의 의미는 전혀 반영되지 않았다)
  
----->word embedding : 수치데이터가 의미를 갖게 하는 것(학습 기반)

padding 방법

In [121]:
#제일 긴 문장 고르기-
#1

import numpy as np

maxlen=np.max([len(d) for d in voca_lst])

for z in range(len(voca_lst)):
    voca_lst[z]=[0]*(maxlen -len(voca_lst[z]))+voca_lst[z]

#   if maxlen < len(voca_lst[z]):
#       maxlen=len(voca_lst[z])
voca_lst

[[0, 0, 0, 11, 4, 1],
 [0, 0, 11, 11, 13, 8],
 [0, 0, 1, 7, 10, 2],
 [0, 0, 0, 1, 11, 5],
 [0, 0, 11, 9, 3, 14],
 [0, 0, 1, 12, 11, 12],
 [1, 1, 1, 11, 11, 11],
 [0, 11, 6, 1, 1, 11]]

In [68]:
len(voca_lst)

8

In [122]:
#2번째 패딩법-keras

voc=sequence.pad_sequences([voca_lst],maxlen)

In [69]:
from keras.models import Model
from keras.layers import Dense, Conv1D, Embedding, Input, GlobalMaxPooling1D,Flatten
from keras import backend as K

In [70]:
x_train=np.array(voca_lst)
y_train=np.array([[0],[1],[1],[0],[0],[1],[0],[0]])
#z_test=np.array()

In [71]:
#간단 모델

K.clear_session()

xInput=Input(batch_shape=(None,6)) #6개 길이 문장 8개
xEmbed=Embedding(input_dim=len(vocabulary)+1, output_dim=4, input_length=6)(xInput) #단어 1개마다 4개의 수치 vector 표현, row는 6개(문장 길이), 8개 리뷰 
#+1안해주면 erorr, padding용을 만들어줌.              #문장의 최대 길이
model=Model(xInput, xEmbed)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 6, 4)              60        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


In [72]:
#진짜 모델
K.clear_session()

xInput=Input(batch_shape=(None,6))
xEmbed=Embedding(input_dim=len(vocabulary)+1, output_dim=4, input_length=6)(xInput)
conv=Conv1D(12,3,activation='relu')(xEmbed)
pool=GlobalMaxPooling1D()(conv)
xOutput=Dense(1,activation='sigmoid', batch_size=1)(pool)

model=Model(xInput, xOutput)
#model.compile(x_train)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 6, 4)              60        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 12)             156       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 12)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 229
Trainable params: 229
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(x_train, y_train, epochs=50, batch_size=1)

Epoch 1/50
8/8 [==============================] - 0s 43ms/step - loss: 0.6945
Epoch 2/50
8/8 [==============================] - 0s 2ms/step - loss: 0.6918
Epoch 3/50
8/8 [==============================] - 0s 0us/step - loss: 0.6894
Epoch 4/50
8/8 [==============================] - 0s 3ms/step - loss: 0.6882
Epoch 5/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6851
Epoch 6/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6836
Epoch 7/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6806
Epoch 8/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6782
Epoch 9/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6760
Epoch 10/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6741
Epoch 11/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6708
Epoch 12/50
8/8 [==============================] - 0s 1ms/step - loss: 0.6685
Epoch 13/50
8/8 [==============================] - 0s 1ms/step - loss: 0

In [74]:
z_text=['dog cat cat human cat cat']
word_vec2=[]
word_vec2.append([diction[z] for v in z_text for z in v.split()])
testset=np.array(word_vec2)

In [75]:
model2 = Model(xInput, xEmbed)
model2.predict(testset)

array([[[ 0.12060194,  0.17085634, -0.04284661, -0.18416691],
        [ 0.02949545, -0.2660891 ,  0.07586849,  0.0193243 ],
        [ 0.02949545, -0.2660891 ,  0.07586849,  0.0193243 ],
        [-0.17096286, -0.1742137 , -0.13638628,  0.1818021 ],
        [ 0.02949545, -0.2660891 ,  0.07586849,  0.0193243 ],
        [ 0.02949545, -0.2660891 ,  0.07586849,  0.0193243 ]]],
      dtype=float32)

out of vocabulary

In [93]:
z_txt='dog cat cat wolf'
                   #out of vocabulary

word_vec3=[]

for word in z_txt.split():
    if word in diction:
        word_vec3.append(diction[word])
    else:
        word_vec3.append(0)

In [100]:
#padding
npad = maxlen - len(word_vec3)

ztext=[]

for i in range(npad):
    ztext.append(0)

In [101]:
for w in word_vec3:
    ztext.append(w)

ztext = np.array(ztext).reshape(1, len(ztext))

In [102]:
ztext

array([[ 0,  0, 11,  1,  1,  0]])

In [103]:
model.predict(ztext)

array([[0.3109434]], dtype=float32)

pad_sequence

In [113]:
from keras.preprocessing import sequence
from keras.models import Sequential

In [114]:
word_vec3

[11, 1, 1, 0]

In [112]:
ztext

array([[ 0,  0, 11,  1,  1,  0]])

In [119]:
ztext=sequence.pad_sequences([word_vec3] ,maxlen=maxlen)

In [120]:
model.predict(ztext)

array([[0.3109434]], dtype=float32)

In [118]:
ztext

array([[ 0,  0, 11,  1,  1,  0]])